In [1]:
import numpy as np

from type_definitions.test_case import TestCase
from type_definitions.test_result import TestResult
from type_definitions.utils import *

In [6]:
cases = generate_cases(10)
results = evaluate_cases(cases)
[r.score for r in results if not r.simulation_failed]

Error evaluating cases: alpha (3.044463299533624) out of bounds
Error evaluating cases: alpha (-4.245617588506868) out of bounds
Error evaluating cases: alpha (5.276349113535417) out of bounds
Error evaluating cases: alpha (2.6973117239339928) out of bounds
Error evaluating cases: velocity (4035.054250484606) out of bounds
Error evaluating cases: alpha (-2.208397697037874) out of bounds


[-500.0, -500.0, -500.0, -500.0]